<a href="https://colab.research.google.com/github/anddennn/IAT360_YogaPoseDetection_CVProject/blob/main/ComputerVisionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b> YOLO-NAS-POSE Yoga Detection Model <b> </h1></center>

Using this youtube video as reference: https://www.youtube.com/watch?v=J83ZvWfxjoA

Code snippets from video's linked google colab folder.

Importing required Libraries

In [ ]:
!pip install super-gradients
!pip install -qq gdown torchinfo

  Using cached super_gradients-3.7.1-py3-none-any.whl.metadata (41 kB)
  Using cached boto3-1.40.59-py3-none-any.whl.metadata (6.6 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached coverage-5.3.1-py3-none-any.whl
  Using cached Sphinx-4.0.3-py3-none-any.whl.metadata (8.0 kB)
  Using cached sphinx_rtd_theme-3.0.2-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached torchmetrics-0.8.0-py3-none-any.whl.metadata (20 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
INFO: pip is looking at multiple versions of super-gradients to determine which version is compatible with other requirements. This could take a while.
  Using cached super_gradients-3.7.0-py3-none-any.whl.metadata (41 kB)
  Using cached super_gradients-3.6.1-py3-none-any.whl.metadata (41 kB)
  Using cached super_gradients-3.6.0-py3-none-any.whl.metadata (40 kB)
  Using cached super_gradients-3.5.0-py3-none-any.whl.metadata (39 kB)
  Using cached super_gradients-3.4.

In [ ]:
!pip install pycocotools==2.0.4 --global-option="build_ext" --global-option="-I/usr/include/python3.10"

DEPRECATION: --build-option and --global-option are deprecated. pip 24.2 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
  Using cached pycocotools-2.0.4.tar.gz (106 kB)
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [ ]:
import json
import os
import random
from pathlib import Path
from typing import Any, List, Tuple, Union

# Third-party imports
import cv2
import gdown
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml
from IPython.display import YouTubeVideo
from sklearn.model_selection import train_test_split

# Colab specific imports
from google.colab.patches import cv2_imshow

# Constants
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from super_gradients.training import models
from super_gradients.common.object_names import Models

yolo_nas_pose = models.get("yolo_nas_pose_l", pretrained_weights="coco_pose").cuda()

ModuleNotFoundError: No module named 'super_gradients'

# Dataset Description






# Data Preprocessing

For preprocessing:
- First manually sorted through the images to remove invalid data (i.e. clipart, children, non-yoga poses, etc.)
- Added more diversity to the dataset
- After annotation, used roboflow to normalize all data into 640x640 jpg images that are auto oriented for better performance
- Augmented data with roboflow to mimic indoor conditions such as adding more low light data or low resolution data.


In [ ]:
from super_gradients.training import Trainer

CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='first_yn_pose_run', ckpt_root_dir=CHECKPOINT_DIR)

## Downloading dataset

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
# fix path to dataset
!cp -r "/content/drive/MyDrive/Dataset" "/content/"

In [26]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1OkaP72I2Cr9MhYbZuXi")
project = rf.workspace("yoga-pose-dataset").project("yoga-poses-yiqyx")
version = project.version(1)
dataset = version.download("coco")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Yoga-Poses-1 in coco:: 100%|██████████| 662/662 [00:00<00:00, 4683.19it/s]


In [27]:
# Get yaml file for standard coco keypoints.
!wget https://raw.githubusercontent.com/Deci-AI/super-gradients/master/src/super_gradients/recipes/dataset_params/coco_pose_estimation_common_dataset_params.yaml

--2025-10-25 21:43:14--  https://raw.githubusercontent.com/Deci-AI/super-gradients/master/src/super_gradients/recipes/dataset_params/coco_pose_estimation_common_dataset_params.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1937 (1.9K) [text/plain]
Saving to: ‘coco_pose_estimation_common_dataset_params.yaml’

coco_pose_estimatio 100%[===================>]   1.89K  --.-KB/s    in 0s      

2025-10-25 21:43:15 (34.8 MB/s) - ‘coco_pose_estimation_common_dataset_params.yaml’ saved [1937/1937]



In [28]:
# @title
def open_file(file_path: str) -> Union[dict, list, None]:
    """
    Opens and reads the content of a JSON or YAML file.

    Parameters:
    file_path (str): The path to the file.

    Returns:
    Union[dict, list, None]: The content of the file parsed to a dictionary or a list,
                             or None if an error occurs.
    """
    try:
        with open(file_path, 'r') as file:
            if file_path.endswith('.json'):
                return json.load(file)
            elif file_path.endswith('.yaml') or file_path.endswith('.yml'):
                return yaml.safe_load(file)
            else:
                raise ValueError(f'Unsupported file format: {file_path}')
    except Exception as e:
        print(f'An error occurred: {e}')
        return None

# Fix paths to dataset ###########

annotations = open_file('/content/drive/MyDrive/Dataset/train/_annotations.coco.json')
config = open_file('/content/drive/MyDrive/Dataset/valid/_annotations.coco.json')

## Annotation File Breakdown:

1. images:
It's a dictionary where each entry maps an ID to a filename (typically an image filename). Each image in the dataset has a unique identifier, and this is a lookup between the ID and the filename.

2. annotations:
This is a list containing 6,117 items. Each item is a dictionary with details related to the annotations for a particular image.
Each annotation contains an image_id, a list of keypoints, and num_keypoints value.

3. categories:
A list of categories for the dataset.
Each category has a:
- supercategory: A broader classification (like 'animal').
- id: A unique identifier for the category.
- name: The name of the category (e.g., 'dog', 'cat', 'sheep').
- keypoints: A list of names for specific keypoints relevant to that category (like 'left_eye', 'right_eye', 'nose', etc.).
- skeleton: A list of pairs, which are connections between keypoints.

In [ ]:
# @title Plotting a sample of images

def plot_random_images(data, image_base_dir="/content/images"):
    """
    Plots 5 random images for each category from the provided dataset.

    Parameters:
    - data: The JSON dataset containing image, annotation, and category details.
    - image_base_dir: The base directory where the images are located.
    """

    # Create a dictionary to map image IDs to filenames
    image_id_to_filename = {image['id']: image['filename'] for image in data['images']}

    # Extracting image_ids for each category
    category_image_ids = {}
    for category in data['categories']:
        category_id = category['id']
        category_name = category['name']
        category_image_ids[category_name] = [anno['image_id'] for anno in data['annotations'] if anno['category_id'] == category_id]

    # Randomly select 5 image_ids for each category
    random_selected_ids = {}
    for category_name, ids in category_image_ids.items():
        random_selected_ids[category_name] = random.sample(ids, min(5, len(ids)))

    # Number of categories
    num_categories = len(random_selected_ids)

    # Create a figure to plot the images
    fig, axes = plt.subplots(num_categories, 5, figsize=(20, num_categories * 3))
    if num_categories == 1:  # If there is only one category, axes will be 1D
        axes = [axes]

    for i, (category_name, ids) in enumerate(random_selected_ids.items()):
        for j, image_id in enumerate(ids):
            # Get the filename using the image_id_to_filename dictionary
            filename = image_id_to_filename.get(image_id, "Image_Not_Found.jpg")

            # Load and plot the image
            img_path = os.path.join(image_base_dir, filename)
            try:
                img = mpimg.imread(img_path)
                axes[i][j].imshow(img)
            except FileNotFoundError:
                axes[i][j].imshow(np.zeros((100, 100, 3)))  # Show an empty image if file is not found
            axes[i][j].axis('off')
            if j == 0:
                axes[i][j].set_title(category_name)

    plt.tight_layout()
    plt.show()

In [ ]:
# Change path for this
plot_random_images(data=annotations, image_base_dir="/content/drive/MyDrive/Dataset/train")

# Datasets & DataLoaders

SuperGradients is fully compatible with PyTorch Datasets and Dataloaders, so you can use your dataloaders as is.

### SuperGradients also provides you with the `AbstractPoseEstimationDataset` class.

This is an abstract class defines a blueprint for datasets related to pose estimation tasks. It's expected that concrete implementations of this class will be created for specific datasets.

- **Inheritance**: It inherits from PyTorch's `Dataset` and `HasPreprocessingParams`.

- **Initialization**:
  - Takes in parameters like `transforms`, `num_joints`, `edge_links`, `edge_colors`, and `keypoint_colors`.
  - Initializes instance variables and constructs a transform pipeline (`KeypointsCompose`).

- **Abstract Methods (`__len__` and `load_sample`)**:
  - These methods are declared but don't have a concrete implementation in this class. Your derived class from this abstract class is expected to provide an implementation for these methods.
  
- **`load_random_sample` Method**:
  - This method is used to fetch a random sample from the dataset. It uses the `__len__` method to get the total number of samples and then randomly selects an index to retrieve using `load_sample`.

- **`__getitem__` Method**:
  - This method retrieves a sample given its index. It then applies the defined transformations on the sample and returns it. This method is crucial for PyTorch's DataLoader to fetch samples during training.

- **`get_dataset_preprocessing_params` Method**:
  - This method defines and returns preprocessing parameters for the dataset. It seems to construct a pipeline of preprocessing steps and their parameters.


In [ ]:
# @title Expand this cell to see how the `AnimalPoseEstimationDataset` is implemented
from super_gradients.common.decorators.factory_decorator import resolve_param
from super_gradients.common.factories.target_generator_factory import TargetGeneratorsFactory
from super_gradients.common.factories.transforms_factory import TransformsFactory
from super_gradients.common.object_names import Datasets
from super_gradients.common.registry import register_dataset
from super_gradients.training.transforms.keypoint_transforms import AbstractKeypointTransform
from super_gradients.training.samples import PoseEstimationSample

from super_gradients.training.datasets.pose_estimation_datasets.abstract_pose_estimation_dataset import AbstractPoseEstimationDataset

from super_gradients.training.datasets.pose_estimation_datasets import YoloNASPoseCollateFN

class PoseEstimationDataset(AbstractPoseEstimationDataset):
    """
    Dataset class for training pose estimation models on Animal Pose dataset.
    """


    @resolve_param("transforms", TransformsFactory())
    def __init__(
        self,
        data_dir: str,
        images_dir: str,
        json_file: str,
        transforms: List[AbstractKeypointTransform],
        edge_links: Union[List[Tuple[int, int]], np.ndarray],
        edge_colors: Union[List[Tuple[int, int, int]], np.ndarray, None],
        keypoint_colors: Union[List[Tuple[int, int, int]], np.ndarray, None],
    ):
        """

        :param data_dir: Root directory of the COCO dataset
        :param images_dir: path suffix to the images directory inside the data_dir
        :param json_file: path suffix to the json file inside the data_dir
        :param include_empty_samples: Not used, but exists for compatibility with COCO dataset config.
        :param target_generator: Target generator that will be used to generate the targets for the model.
            See DEKRTargetsGenerator for an example.
        :param transforms: Transforms to be applied to the image & keypoints
        """
        split_json_file = os.path.join(data_dir, json_file)

        with open(split_json_file, "r") as f:
            json_annotations = json.load(f)


        joints = json_annotations["categories"][0]["keypoints"]
        num_joints = len(joints)

        super().__init__(
            transforms=transforms,
            num_joints=num_joints,
            edge_links=edge_links,
            edge_colors=edge_colors,
            keypoint_colors=keypoint_colors,
        )

        self.num_joints = num_joints
        print(self.num_joints)


        images_and_ids = []

        for image in json_annotations["images"]:
          images_and_ids.append((image["id"], os.path.join(data_dir, images_dir, image["filename"])))
        self.image_ids, self.image_files = zip(*images_and_ids)

        self.annotations = []

        for image_id in self.image_ids:
            keypoints_per_image = []
            bboxes_per_image = []

            image_annotations = [ann for ann in json_annotations["annotations"] if str(ann["image_id"]) == str(image_id)]
            for ann in image_annotations:
                keypoints = np.array(ann["keypoints"]).reshape(self.num_joints, 3)
                x1, y1, x2, y2 = ann["bbox"]

                bbox_xywh = np.array([x1, y1, x2 - x1, y2 - y1])
                keypoints_per_image.append(keypoints)
                bboxes_per_image.append(bbox_xywh)

            keypoints_per_image = np.array(keypoints_per_image, dtype=np.float32).reshape(-1, self.num_joints, 3)
            bboxes_per_image = np.array(bboxes_per_image, dtype=np.float32).reshape(-1, 4)
            annotation = keypoints_per_image, bboxes_per_image
            self.annotations.append(annotation)

    def __len__(self):
        return len(self.image_ids)

    def load_sample(self, index) -> PoseEstimationSample:
        file_path = self.image_files[index]
        gt_joints, gt_bboxes = self.annotations[index]  # boxes in xywh format

        gt_areas = np.array([box[2] * box[3] for box in gt_bboxes], dtype=np.float32)
        gt_iscrowd = np.array([0] * len(gt_joints), dtype=bool)

        image = cv2.imread(file_path, cv2.IMREAD_COLOR)
        mask = np.ones(image.shape[:2], dtype=np.float32)

        return PoseEstimationSample(
            image=image, mask=mask, joints=gt_joints, areas=gt_areas, bboxes_xywh=gt_bboxes, is_crowd=gt_iscrowd, additional_samples=None
        )

In [ ]:
# Directories for annotated files

# Change Paths
train_annotations = open_file('/content/drive/MyDrive/Dataset/train/_annotations.coco.json')
val_annotations = open_file('/content/drive/MyDrive/Dataset/valid/_annotations.coco.json')

## KeyPoint Tranformations

Instantiate the transformations

In [ ]:
# @title Expand this cell to see how the transforms are instantiated
from super_gradients.training.transforms.keypoints import (
    KeypointsHSV,
    KeypointsBrightnessContrast,
    KeypointsMosaic,
    KeypointsRandomAffineTransform,
    KeypointsLongestMaxSize,
    KeypointsPadIfNeeded,
    KeypointsImageStandardize,
    KeypointsImageNormalize,
    KeypointsRemoveSmallObjects
)

# Indexes of keypoints on the flipped image. When doing left-right flip, left hand becomes right hand.
#So this array contains order of keypoints on the flipped image. This is dataset specific and depends on
#how keypoints are defined in dataset.
#keypoints_random_horizontal_flip = KeypointsRandomHorizontalFlip(flip_index=config['flip_indexes'], prob=0.5)

keypoints_hsv = KeypointsHSV(prob=0.5, hgain=20, sgain=20, vgain=20)

keypoints_brightness_contrast = KeypointsBrightnessContrast(prob=0.5,
                                                            brightness_range=[0.8, 1.2],
                                                            contrast_range=[0.8, 1.2]
                                                            )

keypoints_mosaic = KeypointsMosaic(prob=0.8)

keypoints_random_affine_transform = KeypointsRandomAffineTransform(max_rotation=0,
                                                                   min_scale=0.5,
                                                                   max_scale=1.5,
                                                                   max_translate=0.1,
                                                                   image_pad_value=127,
                                                                   mask_pad_value=1,
                                                                   prob=0.75,
                                                                   interpolation_mode=[0, 1, 2, 3, 4]
                                                                   )

keypoints_longest_max_size = KeypointsLongestMaxSize(max_height=640, max_width=640)

keypoints_pad_if_needed = KeypointsPadIfNeeded(min_height=640,
                                               min_width=640,
                                               image_pad_value=[127, 127, 127],
                                               mask_pad_value=1,
                                               padding_mode='bottom_right'
                                               )

keypoints_image_standardize = KeypointsImageStandardize(max_value=255)

# keypoints_image_normalize = KeypointsImageNormalize(mean=[0.485, 0.456, 0.406],
#                                                     std=[0.229, 0.224, 0.225]
#                                                     )

keypoints_remove_small_objects = KeypointsRemoveSmallObjects(min_instance_area=1,
                                                             min_visible_keypoints=1
                                                             )

In [ ]:
train_transforms = [
    keypoints_hsv,
    keypoints_brightness_contrast,
    keypoints_mosaic,
    keypoints_random_affine_transform,
    keypoints_longest_max_size,
    keypoints_pad_if_needed,
    keypoints_image_standardize,
    keypoints_remove_small_objects
]

val_transforms = [
    keypoints_longest_max_size,
    keypoints_pad_if_needed,
    keypoints_image_standardize,
]

In [ ]:
# CHANGE FOR OUR CUSTOM DATA

data_path = "/content/drive/MyDrive/Dataset"

# Create instances of the dataset
train_dataset = PoseEstimationDataset(
    data_dir=data_path,
    images_dir= data_path + '/train',
    json_file= data_path + '/train/keypoint_train.json',
    transforms=train_transforms,
    edge_links = config['edge_links'],
    edge_colors = config['edge_colors'],
    keypoint_colors = config['keypoint_colors']
    )

val_dataset = PoseEstimationDataset(
    data_dir='/content/drive/MyDrive/Dataset',
    images_dir= data_path + '/val',
    json_file= data_path + '/val/keypoint_val.json',
    transforms=val_transforms,
    edge_links = config['edge_links'],
    edge_colors = config['edge_colors'],
    keypoint_colors = config['keypoint_colors']
    )

test_dataset = PoseEstimationDataset(
    data_dir='/content/drive/MyDrive/Dataset',
    images_dir= data_path + '/val',
    json_file= data_path + '/val/keypoint_val.json',
    transforms=val_transforms,
    edge_links = config['edge_links'],
    edge_colors = config['edge_colors'],
    keypoint_colors = config['keypoint_colors']
    )

In [ ]:
from torch.utils.data import DataLoader

# Create dataloaders
train_dataloader_params = {
    'shuffle': True,
    'batch_size': 16,
    'drop_last': True,
    'pin_memory': False,
    'collate_fn': YoloNASPoseCollateFN()
    }

val_dataloader_params = {
    'shuffle': True,
    'batch_size': 16,
    'drop_last': True,
    'pin_memory': False,
    'collate_fn': YoloNASPoseCollateFN()
    }

train_dataloader = DataLoader(train_dataset, **train_dataloader_params)

val_dataloader = DataLoader(val_dataset, **val_dataloader_params)

test_dataloader = DataLoader(test_dataset, **val_dataloader_params)

# Instantiate the model

In [ ]:
yolo_nas_pose = models.get("yolo_nas_pose_l",
                           num_classes=config['num_joints'],
                           pretrained_weights="coco_pose").cuda()

# Training parameters

In [ ]:
# @title Expand this cell to see the training params
from super_gradients.training.models.pose_estimation_models.yolo_nas_pose import YoloNASPosePostPredictionCallback
from super_gradients.training.utils.callbacks import ExtremeBatchPoseEstimationVisualizationCallback, Phase
from super_gradients.training.utils.early_stopping import EarlyStop
from super_gradients.training.metrics import PoseEstimationMetrics

# Note: after next release unwrap all lines wrapped in oc.OmegaConf.create
import omegaconf as oc

post_prediction_callback = YoloNASPosePostPredictionCallback(
  pose_confidence_threshold = 0.01,
  nms_iou_threshold = 0.7,
  pre_nms_max_predictions = 300,
  post_nms_max_predictions = 30,
)

metrics = PoseEstimationMetrics(
  num_joints = config['num_joints'],
  oks_sigmas = config['oks_sigmas'],
  max_objects_per_image = 30,
  post_prediction_callback = post_prediction_callback,
)

visualization_callback = ExtremeBatchPoseEstimationVisualizationCallback(
  keypoint_colors = config["keypoint_colors"],
  edge_colors = config['edge_colors'],
  edge_links = config['edge_links'],
  loss_to_monitor = "YoloNASPoseLoss/loss",
  max = True,
  freq = 1,
  max_images = 1,
  enable_on_train_loader = True,
  enable_on_valid_loader = True,
  post_prediction_callback = post_prediction_callback,
)

early_stop = EarlyStop(
  phase = Phase.VALIDATION_EPOCH_END,
  monitor = "AP",
  mode = "max",
  min_delta = 0.0001,
  patience = 100,
  verbose = True,
)

train_params = {
    "warmup_mode": "LinearBatchLRWarmup",
    "warmup_initial_lr": 1e-8,
    "lr_warmup_epochs": 1,
    "initial_lr": 5e-5,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 5e-3,
    "max_epochs": 5,
    "zero_weight_decay_on_bias_and_bn": True,
    "batch_accumulate": 1,
    "average_best_models": True,
    "save_ckpt_epoch_list": [5, 10, 15, 20],
    "loss": "yolo_nas_pose_loss",
    "criterion_params": {
        "oks_sigmas": config['oks_sigmas'],
        "classification_loss_weight": 1.0,
        "classification_loss_type": "focal",
        "regression_iou_loss_type": "ciou",
        "iou_loss_weight": 2.5,
        "dfl_loss_weight": 0.01,
        "pose_cls_loss_weight": 1.0,
        "pose_reg_loss_weight": 34.0,
        "pose_classification_loss_type": "focal",
        "rescale_pose_loss_with_assigned_score": True,
        "assigner_multiply_by_pose_oks": True,
    },
    "optimizer": "AdamW",
    "optimizer_params": {
        "weight_decay": 0.000001
    },
    "ema": True,
    "ema_params": {
        "decay": 0.997,
        "decay_type": "threshold"
    },
    "mixed_precision": True,
    "sync_bn": False,
    "valid_metrics_list": [metrics],
    "phase_callbacks": [visualization_callback, early_stop],
    "pre_prediction_callback": None,
    "metric_to_watch": "AP",
    "greater_metric_to_watch_is_better": True,
    "_convert_": "all"
}

# Training the model

In [ ]:
# Note, this is training for 10 epochs to demonstrate how to do it -> Change to fewer epochs
trainer.train(model=yolo_nas_pose,
              training_params=train_params,
              train_loader=train_dataloader,
              valid_loader=val_dataloader
              )

NameError: name 'trainer' is not defined

# Get the best trained model

In [ ]:
best_model = models.get('yolo_nas_pose_l',
                        num_classes=config['num_joints'],
                        checkpoint_path="/content/checkpoints/first_yn_pose_run/RUN_20240205_192025_039799/ckpt_best.pth")

# Evaluating the best trained model

In [ ]:
post_prediction_callback = YoloNASPosePostPredictionCallback(
  pose_confidence_threshold = 0.01,
  nms_iou_threshold = 0.7,
  pre_nms_max_predictions = 300,
  post_nms_max_predictions = 30,
)

metrics = PoseEstimationMetrics(
  num_joints = config['num_joints'],
  oks_sigmas = config['oks_sigmas'],
  max_objects_per_image = 30,
  post_prediction_callback = post_prediction_callback,
)

trainer.test(model=best_model,
            test_loader=test_dataloader,
            test_metrics_list=metrics)

# Predicting with the best model

In [ ]:
# Change for our model

img_url = "/content/drive/MyDrive/Dataset/valid"
best_model.predict(img_url, conf=0.20).show()